Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Runtime$\rightarrow$Factory reset runtime) and then **run all cells** (in the menubar, select Runtime$\rightarrow$Run all).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE". Fill out your name below in the `NAME` variable.

In [33]:
NAME = "Ranjani Ramesh"

---

Install required dependency.

In [34]:
!pip install nose

Download the required data.

In [35]:
!wget -c 'https://drive.google.com/uc?export=download&id=1LZDSe3BIVN8nS68WsoS1k_kicTdeq1uF' -O data.zip
!unzip -o data.zip

--2024-09-19 01:48:28--  https://drive.google.com/uc?export=download&id=1LZDSe3BIVN8nS68WsoS1k_kicTdeq1uF
Resolving drive.google.com (drive.google.com)... 173.194.194.138, 173.194.194.113, 173.194.194.102, ...
Connecting to drive.google.com (drive.google.com)|173.194.194.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1LZDSe3BIVN8nS68WsoS1k_kicTdeq1uF&export=download [following]
--2024-09-19 01:48:28--  https://drive.usercontent.google.com/download?id=1LZDSe3BIVN8nS68WsoS1k_kicTdeq1uF&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 173.194.193.132, 2607:f8b0:4001:c0f::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|173.194.193.132|:443... connected.
HTTP request sent, awaiting response... 416 Requested range not satisfiable

    The file is already fully retrieved; nothing to do.

Archive:  data.zip
  inflating: BLOSUM62.txt      

Show the downloaded data

- `./BLOSUM62.txt`
- `./Q2b_reference.txt`


In [36]:
!ls *.txt

BLOSUM62.txt  Q2b_reference.txt


# Sequence Alignment

In bioinformatics, the problem of sequence alignment is reducible to a generalized form of edit distance. Sequence alignment is an instance of the weighted edit distance problem. Here, the cost of the elementary operations may vary according to the symbols involved. This generalization allows us to more accurately model processes in biological sequences that may lead to substitutions, insertions or deletions.

As we saw in lecture, we can store the costs of elementary operations in the form of a scoring matrix $\delta :$ ($\Sigma$ $\cup$ $\{-\})^{2}\rightarrow$ $\mathbb{R}$. The values of this scoring matrix depend on the biological context.

## BLOSUM62

The most widely used scoring matrix in amino acid sequence alignments is known as BLOSUM62. You can read more about BLOSUM here:

* https://www.sciencedirect.com/topics/biochemistry-genetics-and-molecular-biology/blosum
* https://en.wikipedia.org/wiki/BLOSUM
* https://www.nlm.nih.gov/ncbi/workshops/2023-08_BLAST_evol/blast_score.html

The BLOSUM.txt file that came in the HW1_supplement.zip contains the matrix.

Note:
* The `read_blosum62` function is specifically tailored to read only the file above. You should not need to change this function.
* The matrix uses the `*` character for gaps, but we change it here to the `-` character.

In [37]:
def read_blosum62(path):
    """
    Reads in the ncbi's BLOSUM62.txt file and loads the scoring matrix
    into a dictionary.

    :param: path is the full path in the local filesystem at which the .txt file is located
    :return: a dictionary of dictionaries which will hold the cost of various amino acid
    substitutions as defined in BLOSUM62.
    """
    delta = {}
    with open(path, 'r') as f:
        lines = f.readlines()[6:]
        keys = lines[0].split()
        keys[-1] = '-'
        for i, line in enumerate(lines[1:]):
            delta[keys[i]] = {k : int(v) for (k,v) in zip(keys, line.split()[1:])}
    return delta

blosum = read_blosum62('./BLOSUM62.txt')

## Question 1a - Global Alignment - 10 points

Complete the function `global_align` defined below that uses the Needleman-Wunsch algorithm for global sequence alignment. The function should take two strings, and a scoring function, and compute the score of the optimal global alignment. It should return a tuple `(best score, optimal alignment)`.

We have provided a function `traceback_global` that takes the two strings `v` and `w`, and a list of lists called
`pointers`. This list is what you will use to record backpointers while running your sequence alignment algorithm.
Each `pointers[i][j]` will be one of `UP`, `LEFT`, `TOPLEFT` or `ORIGIN`. The `traceback_global` function will insert gaps into the two strings where necessary, according to the backpointer table you pass it, and return the alignment to you.

*Hint: Feel free to define helper functions for this and future questions on this notebook.*

In [38]:
UP = (-1,0)
LEFT = (0, -1)
TOPLEFT = (-1, -1)
ORIGIN = (0, 0)

def traceback_global(v, w, pointers):
    i,j = len(v), len(w)
    new_v = []
    new_w = []
    while True:
        di, dj = pointers[i][j]
        if (di,dj) == LEFT:
            new_v.append('-')
            new_w.append(w[j-1])
        elif (di,dj) == UP:
            new_v.append(v[i-1])
            new_w.append('-')
        elif (di,dj) == TOPLEFT:
            new_v.append(v[i-1])
            new_w.append(w[j-1])
        i, j = i + di, j + dj
        if (i <= 0 and j <= 0):
            break
    return ''.join(new_v[::-1])+'\n'+''.join(new_w[::-1])



def global_align(v, w, delta):
    """
    Returns the score of the maximum scoring alignment of the strings v and w, as well as the actual alignment as
    computed by traceback_global.

    :param: v
    :param: w
    :param: delta
    """

    M = [[0 for j in range(len(w) + 1)] for i in range(len(v) + 1)]
    pointers = [[ORIGIN for j in range(len(w) + 1)] for i in range(len(v) + 1)]
    #Start off by setting the first row and the first column to gap penalitied scores (0, -1, -2, .. -n)
    for i in range(1, len(v) + 1):
        M[i][0] = M[i - 1][0] + delta[v[i - 1]]['-']
        pointers[i][0] = UP
    for j in range(1, len(w) + 1):
        M[0][j] = M[0][j - 1] + delta['-'][w[j - 1]]
        pointers[0][j] = LEFT


    for i in range(1, len(v) + 1):
        for j in range(1, len(w) + 1):
            match_score = M[i - 1][j - 1] + delta[v[i - 1]][w[j - 1]]
            delete_score = M[i - 1][j] + delta[v[i - 1]]['-']
            insert_score = M[i][j - 1] + delta['-'][w[j - 1]]

            M[i][j] = max(match_score, delete_score, insert_score)

            if M[i][j] == match_score:
                pointers[i][j] = TOPLEFT
            elif M[i][j] == delete_score:
                pointers[i][j] = UP
            else:
                pointers[i][j] = LEFT

    score = M[len(v)][len(w)]
    alignment = traceback_global(v, w, pointers)

    return score, alignment

In [39]:
keys = ['A', 'C', 'T', 'G', '-']
delta = {}
for i in range(len(keys)):
    delta[keys[i]] = {k : v for (k,v) in zip(keys, [1 if keys[i] == keys[j]  else -1 for j in range(len(keys))])}

global_align("TAGATA", "GTAGGCTTAAGGTTA", delta)

(-3, '-TA-G----A---TA\nGTAGGCTTAAGGTTA')

## Question 1b -- Global Alignment, Protein Data Bank -- 5 points

https://www.rcsb.org/ provides a database of most known proteins. The search functionality allows us to find information on proteins via their *PDB ID* (https://www.rcsb.org/ [there is a search box in the top right corner]). You can get the exact amino acid sequence of chains in a protein by searching via the PDB ID, then under the *Display Files* dropdown menu, clicking *FASTA Sequence*.

Use the function `global_align` with the BLOSUM62 scoring matrix to find the alignment score for the following pairs of amino acid sequences
* The A-chain of human insulin (#4F1C:A) and the A-chain of bovine insulin (#2ZP6:A). Set the variable `scoreA` below to this value.
* The B-chain of the above proteins. Set the variable `scoreB` below to this value.
* Set the `alignmentA` and `alignmentB` variables to the output of `traceback_global` function for both chains.

In [40]:
scoreA = None
scoreB = None
alignmentA = None
alignmentB = None
# YOUR CODE HERE
human_insulin_A = "GIVEQCCTSICSLYQLENYCN"
human_insulin_B = "FVNQHLCGSHLVEALYLVCGERGFFYTPKT"
bovine_insulin_A = "GIVEQCCASVCSLYQLENYCN"
bovine_insulin_B = "FVNQHLCGSHLVEALYLVCGERGFFYTPKA"
global_align(human_insulin_A, bovine_insulin_A, blosum)
global_align(human_insulin_B, bovine_insulin_B, blosum)

scoreA = global_align(human_insulin_A, bovine_insulin_A, blosum)[0]
scoreB = global_align(human_insulin_B, bovine_insulin_B, blosum)[0]
alignmentA = global_align(human_insulin_A, bovine_insulin_A, blosum)[1]
alignmentB = global_align(human_insulin_A, bovine_insulin_A, blosum)[1]

# raise NotImplementedError()

In [41]:
import nose.tools as Test

# Student test cases (You may add more)
Test.assert_equal(global_align('LRRGEPVY', 'LEKGDTLYILVG', blosum)[0], 5)
Test.assert_equal(global_align('ALYFFT', 'QCASVT', blosum)[0], -2)
keys = ['A', 'C', 'T', 'G', '-']
delta = {}
for i in range(len(keys)):
    delta[keys[i]] = {k : v for (k,v) in zip(keys, [1 if keys[i] == keys[j]  else -1 for j in range(len(keys))])}
Test.assert_equal(global_align('GTTTTATAGC', 'TGGATCGAT', delta)[0], -3)


## Question 2a - Fitting Alignment - 10 points

As we know, next generation sequencing doesn't produce a fully sequenced genome, but rather many short reads. Here, we use fitting alignment.

Complete the `fitting_align` function defined below. The function should take a short read string, a reference genome string, and a scoring matrix, and return the score of the maximum scoring alignment of the short read and all substrings of the reference string, along with the actual fitting alignmnet as computed by the given function `traceback_fitting`. This function is similar to `traceback_global` but also requires you to pass in your dynamic programming table `M`, as well as the starting column for the backtrace `init_j`.

** Note: In the case of ties with respect to determining the starting column for the backtrace, use the rightmost.**

In [42]:
UP = (-1,0)
LEFT = (0, -1)
TOPLEFT = (-1, -1)
ORIGIN = (0, 0)

def traceback_fitting(v, w, M, init_j, pointers):
    i, j = len(v), init_j
    new_v = []
    new_w = []
    while True:
        di, dj = pointers[i][j]
        if (di,dj) == LEFT:
            new_v.append('-')
            new_w.append(w[j-1])
        elif (di,dj) == UP:
            new_v.append(v[i-1])
            new_w.append('-')
        elif (di,dj) == TOPLEFT:
            new_v.append(v[i-1])
            new_w.append(w[j-1])
        i, j = i + di, j + dj
        if (i <= 0):
            break
    return ''.join(new_v[::-1]) + '\n'+''.join(new_w[::-1])

def fitting_align(short, reference, delta):
    """
    Returns the score of the maximum scoring alignment of short and all
    substrings of reference.

    :param: short the shorter of the two strings we are trying to align
    :param: reference the longer string among whose substrings we are doing global alignment
    :param: delta the scoring function for the alphabet of the two strings

    :returns: a tuple (score, alignment)
    """
    M = [[0 for j in range(len(reference)+1)] for i in range(len(short)+1)]
    pointers = [[ORIGIN for j in range(len(reference)+1)] for i in range(len(short)+1)]
    score = None
    init_j = 0

    for i in range(1, len(short) + 1):
        M[i][0] = M[i - 1][0] + delta[short[i - 1]]['-']
        pointers[i][0] = UP

    for j in range(1, len(reference) + 1):
      M[0][j] = 0

    for i in range(1, len(short) + 1):
        for j in range(1, len(reference) + 1):
            match_score = M[i - 1][j - 1] + delta[short[i - 1]][reference[j - 1]]
            delete_score = M[i - 1][j] + delta[short[i - 1]]['-']
            insert_score = M[i][j - 1] + delta['-'][reference[j - 1]]

            M[i][j] = max(match_score, delete_score, insert_score)

            if M[i][j] == match_score:
                pointers[i][j] = TOPLEFT
            elif M[i][j] == delete_score:
                pointers[i][j] = UP
            else:
                pointers[i][j] = LEFT

    #Finding the best score
    best_score = M[len(short)][0]
    init_j = 0
    for j in range(1, len(reference) + 1):
        if M[len(short)][j] >= best_score:  # The = sign accounts for the tie-break
            best_score = M[len(short)][j]
            init_j = j

    score = best_score
    # raise NotImplementedError()
    alignment = traceback_fitting(short,reference,M, init_j,pointers)

    return best_score, alignment

In [43]:
keys = ['A', 'C', 'T', 'G', '-']
delta = {}
for i in range(len(keys)):
    delta[keys[i]] = {k : v for (k,v) in zip(keys, [1 if keys[i] == keys[j]  else -1 for j in range(len(keys))])}

fitting_align("TAGATA", "GTAGGCTTAAGGTTA", delta)

(2, 'T-A-GATA\nTAAGGTTA')

In [44]:
import nose.tools as Test

# Define scoring matrix
keys = ['A', 'C', 'T', 'G', '-']
delta = {}
for i in range(len(keys)):
    delta[keys[i]] = {k : v for (k,v) in zip(keys, [1 if keys[i] == keys[j]  else -1 for j in range(len(keys))])}
delta['-']['-'] = -1

# Student test cases (You can add more)
Test.assert_equal(fitting_align('ACTGCT', 'ACTGTCGTACGTGTACGTGCTATTACGATTCGGATGCATTGTGCATTTGGGCGATCTTATTCTTATC', delta)[0],\
                  5)
Test.assert_equal(fitting_align('PPPVP','MHHHHHHSSPIDPPGKPVPLNITRHTVTLKWAKPEYTGGFKITSYIVEKRDLPNGRWLKANFSNILENEFTVSGLTEDAA\
YEFRVIAKNAAGAISPPSEPSDAITCRDDVEA', blosum)[0], 24)


## Question 2b - Fitting Alignment - 5 points

After writing the function use it to align the reference sequence `./Q2b_reference.txt` and the short read provided below. The reference is a partial version of the human Y-chromosome reference found at [https://www.ncbi.nlm.nih.gov/nuccore/NC_000024.10?report=fasta](https://www.ncbi.nlm.nih.gov/nuccore/NC_000024.10?report=fasta).

Set the variable `score_fitting` to the fitting alignment score, and set the `alignment_fitting` to the output of the `traceback_fitting` function. You can use the scoring matrix defined below.

** Note: The reference sequence is very large, and so the fitting alignment will take a few minutes. Make sure you use dynamic programming to implement your alignment functions, or the cell will never finish. **

In [45]:
file_path = "/content/Q2b_reference.txt"

with open(file_path, 'r') as file:
    file_content = ''
    line = file.readline()
    # print(line)
    line = file.readline()
    while line:
        file_content += line
        line = file.readline()

file_content = file_content.replace('\n', '')
print(list(set(file_content)))


['G', 'T', 'A', 'C']


In [46]:
short = 'CCCAAAGCCCGGTGGAAAAAGACAGCCCGGCGAAGACCCAAAGCCCAGCCCAAGACACCTCAATCATGTCGAGAAATAACGCAGATACAGGCAGAGTTCTTGCCTTACCAGAGCACAAGAAGAAGCGCAAGGGAAAC'
keys = ['A', 'C', 'T', 'G', '-']
delta_fitting = {}
for i in range(len(keys)):
    delta_fitting[keys[i]] = {k : v for (k,v) in zip(keys, [1 if keys[i] == keys[j]  else -1 for j in range(len(keys))])}
delta_fitting['-']['-'] = -1

# Read the reference sequence, and call your fitting alignment function
# YOUR CODE HERE
tup = fitting_align(short, file_content, delta_fitting)
score_fitting = tup[0]
alignment_fitting = tup[1]

print(score_fitting, alignment_fitting)


137 CCCAAAGCCCGGTGGAAAAAGACAGCCCGGCGAAGACCCAAAGCCCAGCCCAAGACACCTCAATCATGTCGAGAAATAACGCAGATACAGGCAGAGTTCTTGCCTTACCAGAGCACAAGAAGAAGCGCAAGGGAAAC
CCCAAAGCCCGGTGGAAAAAGACAGCCCGGCGAAGACCCAAAGCCCAGCCCAAGACACCTCAATCATGTCGAGAAATAACGCAGATACAGGCAGAGTTCTTGCCTTACCAGAGCACAAGAAGAAGCGCAAGGGAAAC


## Question 2c - 5 points

Find the name of the gene the short read in the question above originated from. Save your answer in variable `ANSWER` below.

*Hint : There is an online tool that will do this for you*

In [47]:
# YOUR CODE HERE
ANSWER="TGIF2LY"
# raise NotImplementedError()

## Question 3 - Local Alignment - 15 points

Finally, we use local alignment for things like aligning functional units of proteins called domains.

Use the Smith-Waterman algorithm to complete the function `local_align` defined below that takes two strings `v` and `w` and a scoring matrix, and returns the maximum scoring global alignment across all substrings of `v` and `w`, as well as the alignment string produced by `traceback_local`. As with the above functions, you must record backpointers. This function is very similar to `traceback_fitting`, except it also takes the starting cell for the back trace in the form of coordinates `init_i, init_j`.

** Note: In the case of ties with respect to determining the starting column for the backtrace, use the bottom-right most cell.**

In [48]:
UP = (-1,0)
LEFT = (0, -1)
TOPLEFT = (-1, -1)
ORIGIN = (0, 0)

def traceback_local(v, w, M, init_i, init_j, pointers):
    i,j = init_i, init_j
    new_v = []
    new_w = []
    while True:
        di, dj = pointers[i][j]
        if (di,dj) == LEFT:
            new_v.append('-')
            new_w.append(w[j-1])
        elif (di,dj) == UP:
            new_v.append(v[i-1])
            new_w.append('-')
        elif (di,dj) == TOPLEFT:
            new_v.append(v[i-1])
            new_w.append(w[j-1])
        i, j = i + di, j + dj
        if (M[i][j] == 0):
            break
    return ''.join(new_v[::-1]) + '\n'+''.join(new_w[::-1])

def local_align(v, w, delta):
    """
    Returns the score of the maximum scoring alignment of all possible substrings of v and w.

    :param: v
    :param: w
    :param: delta
    """
    M = [[0 for j in range(len(w)+1)] for i in range(len(v)+1)]
    pointers = [[ORIGIN for j in range(len(w)+1)] for i in range(len(v)+1)]
    max_score = 0
    init_i, init_j = 0, 0

    for i in range(1, len(v)+1):
        for j in range(1, len(w)+1):

            matching = M[i-1][j-1] + delta[v[i - 1]][w[j - 1]]
            delete = M[i-1][j] + delta[v[i - 1]]['-']
            insert = M[i][j-1] + delta['-'][w[j - 1]]

            M[i][j] = max(0, matching, delete, insert)

            if M[i][j] == matching:
                pointers[i][j] = TOPLEFT
            elif M[i][j] == delete:
                pointers[i][j] = UP
            elif M[i][j] == insert:
                pointers[i][j] = LEFT
            else:
                pointers[i][j] = ORIGIN

            if M[i][j] >= max_score:
                max_score = M[i][j]
                init_i, init_j = i, j

    alignment = traceback_local(v, w, M, init_i, init_j, pointers)

    return max_score, alignment

In [49]:
keys = ['A', 'C', 'T', 'G', '-']
delta = {}
for i in range(len(keys)):
    delta[keys[i]] = {k : v for (k,v) in zip(keys, [1 if keys[i] == keys[j]  else -1 for j in range(len(keys))])}

local_align("TAGATA", "GTAGGCTTAAGGTTA", delta)

(3, 'TAG\nTAG')

Use your `local_align` function to find the common domain between the two amino acid sequences given below.
* Set the `score_local` variable to the output of your function.
* Set the `alignment_local` variable to the output of the `traceback_local` function.

In [50]:
LTK_MOUSE = "MGCSHRLLLWLGAAGTILCSNSEFQTPFLTPSLLPVLVLNSQEQKVTPTP\
SKLEPASLPNPLGTRGPWVFNTCGASGRSGPTQTQCDGAYTGSSVMVTVG\
AAGPLKGVQLWRVPDTGQYLISAYGAAGGKGAQNHLSRAHGIFLSAVFFL\
RRGEPVYILVGQQGQDACPGGSPESQLVCLGESGEHATTYGTERIPGWRR\
WAGGGGGGGGATSIFRLRAGEPEPLLVAAGGGGRSYRRRPDRGRTQAVPE\
RLETRAAAPGSGGRGGAAGGGSGWTSRAHSPQAGRSPREGAEGGEGCAEA\
WAALRWAAAGGFGGGGGACAAGGGGGGYRGGDTSESDLLWADGEDGTSFV\
HPSGELYLQPLAVTEGHGEVEIRKHPNCSHCPFKDCQWQAELWTAECTCP\
EGTELAVDNVTCMDLPTTASPLILMGAVVAALALSLLMMCAVLILVNQKC\
QGLWGTRLPGPELELSKLRSSAIRTAPNPYYCQVGLSPAQPWPLPPGLTE\
VSPANVTLLRALGHGAFGEVYEGLVTGLPGDSSPLPVAIKTLPELCSHQD\
ELDFLMEALIISKFSHQNIVRCVGLSFRSAPRLILLELMSGGDMKSFLRH\
SRPHPGQLAPLTMQDLLQLAQDIAQGCHYLEENHFIHRDIAARNCLLSCS\
GASRVAKIGDFGMARDIYQASYYRKGGRTLLPVKWMPPEALLEGLFTSKT\
DSWSFGVLLWEIFSLGYMPYPGHTNQEVLDFIATGNRMDPPRNCPGPVYR\
IMTQCWQHQPELRPDFGSILERIQYCTQDPDVLNSPLPVEPGPILEEEEA\
SRLGNRSLEGLRSPKPLELSSQNLKSWGGGLLGSWLPSGLKTLKPRCLQP\
QNIWNPTYGSWTPRGPQGEDTGIEHCNGSSSSSIPGIQ"

ATK_MOUSE = "MGAAGFLWLLPPLLLAAASYSGAATDQRAGSPASGPPLQPREPLSYSRLQ\
RKSLAVDFVVPSLFRVYARDLLLPQPRSPSEPEAGGLEARGSLALDCEPL\
LRLLGPLPGISWADGASSPSPEAGPTLSRVLKGGSVRKLRRAKQLVLELG\
EETILEGCIGPPEEVAAVGILQFNLSELFSWWILHGEGRLRIRLMPEKKA\
SEVGREGRLSSAIRASQPRLLFQIFGTGHSSMESPSETPSPPGTFMWNLT\
WTMKDSFPFLSHRSRYGLECSFDFPCELEYSPPLHNHGNQSWSWRHVPSE\
EASRMNLLDGPEAEHSQEMPRGSFLLLNTSADSKHTILSPWMRSSSDHCT\
LAVSVHRHLQPSGRYVAQLLPHNEAGREILLVPTPGKHGWTVLQGRVGRP\
ANPFRVALEYISSGNRSLSAVDFFALKNCSEGTSPGSKMALQSSFTCWNG\
TVLQLGQACDFHQDCAQGEDEGQLCSKLPAGFYCNFENGFCGWTQSPLSP\
HMPRWQVRTLRDAHSQGHQGRALLLSTTDILASEGATVTSATFPAPMKNS\
PCELRMSWLIRGVLRGNVSLVLVENKTGKEQSRTVWHVATDEGLSLWQHT\
VLSLLDVTDRFWLQIVTWWGPGSRATVGFDNISISLDCYLTISGEEKMSL\
NSVPKSRNLFEKNPNKESKSWANISGPTPIFDPTVHWLFTTCGASGPHGP\
TQAQCNNAYQNSNLSVVVGSEGPLKGVQIWKVPATDTYSISGYGAAGGKG\
GKNTMMRSHGVSVLGIFNLEKGDTLYILVGQQGEDACPRANQLIQKVCVG\
ENNVIEEEIRVNRSVHEWAGGGGGGGGATYVFKMKDGVPVPLIIAAGGGG\
RAYGAKTETFHPERLESNSSVLGLNGNSGAAGGGGGWNDNTSLLWAGKSL\
LEGAAGGHSCPQAMKKWGWETRGGFGGGGGGCSSGGGGGGYIGGNAASNN\
DPEMDGEDGVSFISPLGILYTPALKVMEGHGEVNIKHYLNCSHCEVDECH\
MDPESHKVICFCDHGTVLADDGVSCIVSPTPEPHLPLSLILSVVTSALVA\
ALVLAFSGIMIVYRRKHQELQAMQMELQSPEYKLSKLRTSTIMTDYNPNY\
CFAGKTSSISDLKEVPRKNITLIRGLGHGAFGEVYEGQVSGMPNDPSPLQ\
VAVKTLPEVCSEQDELDFLMEALIISKFNHQNIVRCIGVSLQALPRFILL\
ELMAGGDLKSFLRETRPRPNQPTSLAMLDLLHVARDIACGCQYLEENHFI\
HRDIAARNCLLTCPGAGRIAKIGDFGMARDIYRASYYRKGGCAMLPVKWM\
PPEAFMEGIFTSKTDTWSFGVLLWEIFSLGYMPYPSKSNQEVLEFVTSGG\
RMDPPKNCPGPVYRIMTQCWQHQPEDRPNFAIILERIEYCTQDPDVINTA\
LPIEYGPVVEEEEKVPMRPKDPEGMPPLLVSPQPAKHEEASAAPQPAALT\
APGPSVKKPPGAGAGAGAGAGAGPVPRGAADRGHVNMAFSQPNPPPELHK\
GPGSRNKPTSLWNPTYGSWFTEKPAKKTHPPPGAEPQARAGAAEGGWTGP\
GAGPRRAEAALLLEPSALSATMKEVPLFRLRHFPCGNVNYGYQQQGLPLE\
ATAAPGDTMLKSKNKVTQPGP"

tup = local_align(LTK_MOUSE, ATK_MOUSE, blosum)
score_local, alignment_local = tup[0], tup[1]

# YOUR CODE HERE
# raise NotImplementedError()


In [51]:
import nose.tools as Test
# Student test cases (You may add more)
keys = ['A', 'C', 'T', 'G', '-']
delta_local = {}
for i in range(len(keys)):
    delta_local[keys[i]] = {k : v for (k,v) in zip(keys, [1 if keys[i] == keys[j]  else -1 for j in range(len(keys))])}
delta_local['-']['-'] = -1
Test.assert_equal(local_align("TACGTGACGTCTATCAT", "TTGTGCATGTATCTGAC", delta_local)[0], 6)
Test.assert_equal(local_align("ACTGATTTCGATGCTGTACGTGACGTACGTTTATTCTATCAT", "TCTGACTGTGACTATACTATTGTGCATGTATCTGACTAGCTAG", delta_local)[0], 13)
